### Import packages

In [ ]:
import pandas as pd
import csv
import chardet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

### Read data

In [ ]:
with open("classifier_output_latest.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}

In [ ]:
rows = []
count = 0
with open("classifier_output_latest.csv", 'r',encoding = 'Windows-1252') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    for row in csvreader:
        rows.append(row)
        count += 1
    print(count)

18441


In [ ]:
df = pd.DataFrame(rows)
df = df.iloc[:,:11]
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,"I recently traded in my 2017 Honda HR-V in ""Ba...",5,1,1,0,1,1,0,0,1,0
1,Recently purchased a Taos in the base trim (S)...,5,0,0,1,0,0,0,0,1,0
2,This car feels premium and looks handsome. It...,5,1,0,1,0,1,0,0,0,0
3,"Bought the White SEL, love everything about it...",1,0,0,0,0,-1,0,-1,0,-1
4,"The FWD Taos S is a sporty, fun drive. It look...",5,1,1,1,1,0,0,0,0,0


In [ ]:
df.columns = ['Review','Total_star','Value','Size','Comfort_Drive',\
              'Interior','Appearance_Exterior','Power_Performance','Safety','Mpg_Efficiency','Maintanence']
df.head()

,Review,Total_star,Value,Size,Comfort_Drive,Interior,Appearance_Exterior,Power_Performance,Safety,Mpg_Efficiency,Maintanence
0,"I recently traded in my 2017 Honda HR-V in ""Ba...",5,1,1,0,1,1,0,0,1,0
1,Recently purchased a Taos in the base trim (S)...,5,0,0,1,0,0,0,0,1,0
2,This car feels premium and looks handsome. It...,5,1,0,1,0,1,0,0,0,0
3,"Bought the White SEL, love everything about it...",1,0,0,0,0,-1,0,-1,0,-1
4,"The FWD Taos S is a sporty, fun drive. It look...",5,1,1,1,1,0,0,0,0,0


### Clean data

In [ ]:
df.dtypes

Review                 object
Total_star             object
Value                  object
Size                   object
Comfort_Drive          object
Interior               object
Appearance_Exterior    object
Power_Performance      object
Safety                 object
Mpg_Efficiency         object
Maintanence            object
dtype: object

In [ ]:
df["Total_star"].value_counts()

5                                                                                                                  10748
4                                                                                                                   3155
1                                                                                                                   1657
3                                                                                                                   1638
2                                                                                                                   1234
0                                                                                                                      3
 be warned that the mileage figures on the sticker are wishful thinking at best.   With roughly a 2/3 city             1
 I will never be able to enjoy an ICE car as a daily driver. The instant torque of the Model 3 is extraordinary        1
 super car. When this car got 4 

In [ ]:
df.describe()

,Review,Total_star,Value,Size,Comfort_Drive,Interior,Appearance_Exterior,Power_Performance,Safety,Mpg_Efficiency,Maintanence
count,18441,18441,18441,18441,18441,18441,18441,18441,18441,18441,18441
unique,18158,12,9,9,9,8,7,7,6,6,6
top,Great car,5,0,0,0,0,0,0,0,0,0
freq,7,10748,16501,16201,11699,14033,16625,14411,17089,15228,15089


In [ ]:
indexs = []
for i in range(len(df)):
    if not df["Total_star"][i] in ['1','2','3','4','5']:
        indexs.append(i)
indexs

[845, 4893, 6270, 7624, 7809, 11533, 12201, 13836, 13966]

In [ ]:
df.iloc[indexs,:]

,Review,Total_star,Value,Size,Comfort_Drive,Interior,Appearance_Exterior,Power_Performance,Safety,Mpg_Efficiency,Maintanence
845,Very disappointed with the hollowing noice as ...,you know the buzz says that the 140hp engine ...,which in my mind is a silly reason to downgra...,you shouldn't be roaring to 60 in a second. Y...,all my sporting activities,traveling with my partner and my dogs,and feeling that they are safe and comfortabl...,yes,because again,it drives easy and the Sport mode allows me t...,and I don't need to take a mountainside in th...
4893,The 2019 CX 9 Signature is two stars shy of be...,former football player and avid weightlifter....,especially in the upper trim levels,"Grand Touring and Signature.""",5,1,0,1,0,0,0
6270,I love my new XT4. It rides smooth and is very...,0,0,1,0,0,0,0,0,0,
7624,Lexus has a big problem with the interiors of ...,0,0,0,0,0,0,0,0,-1,
7809,As a 2013 A8L 4.0 owner and 2015 Q7 TDI... I d...,be warned that the mileage figures on the sti...,1/3 highway split,our overall mileage is 14.7 mpg. We haven't...,I can't imagine how low our mileage would be....,we accelerate slowly,we let off the gas well in advance of a stop ...,etc.,etc. And the result of all that careful dri...,mostly city driving,was 11.2 mpg. We weren't expecting to win a...
11533,They Crippled Entune 3.0 radio/infomation syst...,and then start the app on Entune. If you go ...,functionality goes away until back in network...,you need to remember to unplug and take your ...,you will only see navigation on a V6 with int...,4,0,0,0,0,0
12201,Thank you Chrysler!!!!?€?5,0,0,0,0,0,0,0,0,0,
13836,I love this car. My daily drivers were a Porsc...,super car. When this car got 4 wheel drive,I knew I had to get one. This car is fully lo...,class,acceleration,sound,handling,technology and fuel economy in a 600 horsepow...,you can get this beast fixed anywhere. This i...,4,0
13966,"Tesla is a game changer, ICE vehicles are rapi...",I will never be able to enjoy an ICE car as a...,the range for the long range is more than eno...,for the upgraded version at least. I never ex...,lose windows,or a problem with the trunk. The app has ever...,including on demand servicing. Highly suggest...,5,0,0,0


In [ ]:
# Drop the rows shown before
df.drop(df.index[indexs],inplace=True)

In [ ]:
df.describe()

,Review,Total_star,Value,Size,Comfort_Drive,Interior,Appearance_Exterior,Power_Performance,Safety,Mpg_Efficiency,Maintanence
count,18432,18432,18432,18432,18432,18432,18432,18432,18432,18432,18432
unique,18149,5,3,3,3,3,3,3,3,3,3
top,Great car,5,0,0,0,0,0,0,0,0,0
freq,7,10748,16498,16199,11696,14030,16620,14407,17083,15223,15085


### Predictors and Response

In [ ]:
y = df['Total_star']
X = df.iloc[:,2:11]

In [ ]:
y.dtypes

dtype('O')

In [ ]:
y = y.astype(int)
y.dtypes

dtype('int32')

In [ ]:
X.dtypes

Value                  object
Size                   object
Comfort_Drive          object
Interior               object
Appearance_Exterior    object
Power_Performance      object
Safety                 object
Mpg_Efficiency         object
Maintanence            object
dtype: object

In [ ]:
X = X.astype(int)
X.dtypes

Value                  int32
Size                   int32
Comfort_Drive          int32
Interior               int32
Appearance_Exterior    int32
Power_Performance      int32
Safety                 int32
Mpg_Efficiency         int32
Maintanence            int32
dtype: object

### Regression models

##### Linear Regression

In [ ]:
def linear_Regression(X,y):    
    model = LinearRegression()
    model.fit(X,y)
    y_pred = model.predict(X)
    R2 = model.score(X,y)
    MAE = mean_absolute_error(y, y_pred)
    print("The R square of Linear Regression model is:", R2)
    print("The MAE of Linear Regression model is:", MAE)

##### Decision Tree

In [ ]:
def DecisionTree_Regressor(X,y):
    model = DecisionTreeRegressor(random_state=0)
    params = {'max_depth':[i for i in range(3,30)]}
    grid = GridSearchCV(model, param_grid = params, cv = 5)
    grid.fit(X,y)
    print(grid.best_params_)

    y_pred = grid.predict(X)
    R2 = grid.score(X,y)
    MAE = mean_absolute_error(y, y_pred)
    print("The R square of Drcision Tree Regression model is:", R2)
    print("The MAE of Decision Tree Regression model is:", MAE)

##### Random Forest

In [ ]:
def Random_Forest(X,y):
    model = RandomForestRegressor(random_state=0)
    params = {'n_estimators':[75,100,125],
              'max_features':['auto', 'sqrt'], 
              'max_depth':[i for i in range(3,10)]}
    grid = GridSearchCV(model, param_grid = params, cv = 5)
    grid.fit(X.values,y)
    print(grid.best_params_)
    
    y_pred = grid.predict(X.values)
    R2 = grid.score(X.values,y)
    MAE = mean_absolute_error(y, y_pred)
    print("The R square of Random Forest Regression model is:", R2)
    print("The MAE of Random Forest Regression model is:", MAE)

##### GBDT

In [ ]:
def GBDT(X,y):
    model = GradientBoostingRegressor(random_state=0)
    model.fit(X.values,y)

    y_pred = model.predict(X.values)
    R2 = model.score(X.values,y)
    MAE = mean_absolute_error(y, y_pred)
    print("The R square of GBDT model is:", R2)
    print("The MAE of GBDT model is:", MAE)

In [ ]:
linear_Regression(X,y)

The R square of Linear Regression model is: 0.2424167055920029
The MAE of Linear Regression model is: 0.906495280953157


In [ ]:
DecisionTree_Regressor(X,y)

{'max_depth': 8}
The R square of Drcision Tree Regression model is: 0.2924250585908751
The MAE of Decision Tree Regression model is: 0.8473022210165743


In [ ]:
Random_Forest(X,y)

{'max_depth': 9, 'max_features': 'sqrt', 'n_estimators': 125}
The R square of Random Forest Regression model is: 0.29800171734887315
The MAE of Random Forest Regression model is: 0.8499415838481952


In [ ]:
GBDT(X,y)

The R square of GBDT model is: 0.2922054904829058
The MAE of GBDT model is: 0.8538103411646362


In [ ]:
# df.to_csv('classifier_output_updated.csv')

# Result for Decision Tree

In [ ]:
model = DecisionTreeRegressor(random_state = 0, max_depth=8)
model.fit(X,y)
model.feature_importances_

array([0.02866135, 0.01571649, 0.17796978, 0.03680861, 0.00689695,
       0.0301279 , 0.0081986 , 0.02990482, 0.66571549])

In [ ]:
df_importance = pd.DataFrame(100*model.feature_importances_,
                   index = X.columns,
                   columns=['importance'])
df_importance = df_importance.sort_values(by = 'importance',axis=0,
                      ascending=False)
df_importance

,importance
Maintanence,66.571549
Comfort_Drive,17.796978
Interior,3.680861
Power_Performance,3.012790
Mpg_Efficiency,2.990482
Value,2.866135
Size,1.571649
Safety,0.819860
Appearance_Exterior,0.689695


# Result for Random Forest

In [ ]:
model = RandomForestRegressor(random_state = 0, max_depth = 9, max_features = 'sqrt', n_estimators = 125)
model.fit(X,y)
model.feature_importances_

array([0.03132739, 0.03077934, 0.21066939, 0.04497978, 0.01406341,
       0.04005384, 0.01362306, 0.03748552, 0.57701826])

In [ ]:
df_importance = pd.DataFrame(100*model.feature_importances_,
                   index = X.columns,
                   columns=['importance'])
df_importance = df_importance.sort_values(by = 'importance',axis=0,
                      ascending=False)
df_importance

,importance
Maintanence,57.701826
Comfort_Drive,21.066939
Interior,4.497978
Power_Performance,4.005384
Mpg_Efficiency,3.748552
Value,3.132739
Size,3.077934
Appearance_Exterior,1.406341
Safety,1.362306


### Validation using the ratings on Edmunds (ignorable)

In [ ]:
df1 = pd.read_csv("data_cleaned.csv")

In [ ]:
X = df1.iloc[:,8:15]
y = df1["Total_star"]

In [ ]:
y.value_counts()

5    10756
4     3159
1     1658
3     1639
2     1237
Name: Total_star, dtype: int64

In [ ]:
X.describe()

,Safety,Technology,Performance,Interior,Comfort,Reliability,Value
count,18449.000000,18449.000000,18449.000000,18449.000000,18449.000000,18449.000000,18449.000000
mean,3.412109,3.226787,3.468643,3.454334,3.479647,3.100005,3.060274
std,2.113483,2.061752,1.966299,2.004870,1.975602,2.193611,2.106794
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,2.000000,2.000000,2.000000,0.000000,0.000000
50%,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [ ]:
DecisionTree_Regressor(X,y)

{'max_depth': 7, 'max_features': 'auto'}
The R square of Drcision Tree Regression model is: 0.6680110866675395
The MAE of Decision Tree Regression model is: 0.4726547338070822
